# Two-Room Scene Viewer

Interactive 3D viewer for the two-room scene with concrete walls and doorway.

**Scene Details:**
- Two 20m x 40m rooms separated by concrete dividing wall at x=20m
- Doorway: 1.3m wide x 2m high (centered at y=20m, z=1-3m)
- Wall material: `itu_concrete`, thickness=10cm
- External walls at boundaries: x=0, x=40m, y=0, y=40m
- Floor at z=0, walls extend to z=3m (no ceiling)

**Viewer Controls:**
- **Mouse left**: Rotate
- **Scroll wheel**: Zoom
- **Mouse right**: Pan
- **Alt + click**: Pick point coordinates

In [ ]:
from sionna.rt import load_scene
import drjit as dr

# Load the two-room scene from ../../scenes/
SCENE_FILE = "../../scenes/two_rooms.xml"

# Use merge_shapes=False to keep individual surfaces separate (for inspection)
scene = load_scene(SCENE_FILE, merge_shapes=False)

# List all scene objects (surfaces) with their IDs, positions, and sizes
print("Scene Objects (surfaces):")
print("=" * 80)
print(f"{'ID':<25} {'Center (x,y,z)':<25} {'Size (w,h,d)':<25}")
print("-" * 80)

for name, obj in scene.objects.items():
    # Get center position (axis-aligned bounding box center)
    # Use drjit to extract scalar values
    pos = obj.position
    px, py, pz = pos[0][0], pos[1][0], pos[2][0]
    center = f"({px:.2f}, {py:.2f}, {pz:.2f})"
    
    # Get bounding box from Mitsuba mesh to calculate size
    bbox = obj.mi_mesh.bbox()
    size_x = bbox.max[0] - bbox.min[0]
    size_y = bbox.max[1] - bbox.min[1]
    size_z = bbox.max[2] - bbox.min[2]
    size = f"({size_x:.2f}, {size_y:.2f}, {size_z:.2f})"
    
    print(f"{name:<25} {center:<25} {size:<25}")

print("-" * 80)
print(f"Total: {len(scene.objects)} objects")
print("\nTip: Use Alt+click in preview to get coordinates of any point")

scene.preview()

## View with Axis Markers

Add TX/RX devices at origin and along axes as visual markers:

In [ ]:
from sionna.rt import Transmitter, Receiver, PlanarArray

# Reload scene to add axis markers
scene_with_axes = load_scene(SCENE_FILE, merge_shapes=False)

# Configure minimal antenna arrays
scene_with_axes.tx_array = PlanarArray(num_rows=1, num_cols=1, vertical_spacing=0.5,
                                        horizontal_spacing=0.5, pattern="iso", polarization="V")
scene_with_axes.rx_array = PlanarArray(num_rows=1, num_cols=1, vertical_spacing=0.5,
                                        horizontal_spacing=0.5, pattern="iso", polarization="V")

# Add markers: Origin (green=TX), and axis endpoints (blue=RX)
# Origin marker
scene_with_axes.add(Transmitter(name="origin", position=[0, 0, 0]))

# X-axis marker (1m along X)
scene_with_axes.add(Receiver(name="x_axis_1m", position=[1, 0, 0]))

# Y-axis marker (1m along Y) 
scene_with_axes.add(Receiver(name="y_axis_1m", position=[0, 1, 0]))

# Z-axis marker (1m along Z)
scene_with_axes.add(Receiver(name="z_axis_1m", position=[0, 0, 1]))

print("Axis markers added:")
print("  Green (TX) = Origin (0,0,0)")
print("  Blue (RX)  = X-axis (1,0,0), Y-axis (0,1,0), Z-axis (0,0,1)")

scene_with_axes.preview()

## View with Clipping

Use `clip_at` to cut away upper walls and see inside the rooms:

In [ ]:
# Preview with upper walls clipped away (z=2.0 cuts below 3m ceiling)
scene.preview(clip_at=2.0)

## Add Devices and View Paths

Add transmitters/receivers in different rooms and visualize propagation paths.

**About Path Tracing Through Walls:**

When viewing paths, you may notice some paths go **through** the concrete dividing wall. This is **expected behavior** and physically accurate:

### Why Paths Go Through Concrete

1. **Refraction is enabled by default** - `PathSolver()` traces transmission (refraction) paths through materials unless explicitly disabled with `refraction=False`

2. **Slab transmission model** - Sionna models the 10cm concrete wall as a slab where:
   - Rays go straight through without angular deflection (Snell's law bending ignored)
   - Power attenuation is computed based on material properties and thickness
   - Multiple internal reflections inside the slab are accounted for

3. **Physical accuracy at 5.18 GHz**:
   - Wavelength ≈ 5.8 cm
   - 10cm wall ≈ 1.7 wavelengths thick
   - **Some RF energy WILL penetrate**, but with **40-60 dB attenuation**
   - ITU-R P.2040-3 concrete parameters: εᵣ = 5.24, σ ≈ 0.15 S/m at 5.18 GHz

### What to Expect

- **Through-wall paths**: Highly attenuated (-80 to -100 dB)
- **Through-doorway LOS**: Much stronger (-40 to -60 dB)
- **Reflections/diffraction**: Intermediate attenuation (-60 to -80 dB)

### How to Verify

Check path powers to confirm through-wall paths are properly attenuated (see code below).

### To Block Transmission Through Walls

If you want to completely block transmission:
- **Option A**: Disable refraction globally: `PathSolver(refraction=False)`
- **Option B**: Use metal material: `itu_metal` (σ = 10⁷ S/m blocks transmission)

In [ ]:
from sionna.rt import Transmitter, Receiver, PlanarArray, PathSolver
import numpy as np

# Reload scene fresh (merge_shapes=False to keep individual surfaces)
scene = load_scene(SCENE_FILE, merge_shapes=False)

# Configure antenna arrays
scene.tx_array = PlanarArray(num_rows=1, num_cols=1, vertical_spacing=0.5,
                              horizontal_spacing=0.5, pattern="iso", polarization="V")
scene.rx_array = PlanarArray(num_rows=1, num_cols=1, vertical_spacing=0.5,
                              horizontal_spacing=0.5, pattern="iso", polarization="V")

# Set frequency (important for ITU material properties)
scene.frequency = 5.18e9  # 5.18 GHz (WiFi 5)

# Add TX in Room 1 (left room), RX in Room 2 (right room)
# This will show paths through doorway AND through concrete wall
tx = Transmitter(name="tx", position=[10, 20, 1.5])  # Center of Room 1
rx = Receiver(name="rx", position=[30, 20, 1.5])     # Center of Room 2
scene.add(tx)
scene.add(rx)

# Compute paths (refraction enabled by default)
solver = PathSolver()
paths = solver(scene)

# Preview with paths (clip to see inside)
scene.preview(paths=paths, clip_at=2.0)

## Analyze Path Powers

Verify that through-wall paths are highly attenuated compared to through-doorway paths:

In [ ]:
# Extract path data with NumPy conversion
# cir() returns (a, tau) where shapes depend on synthetic_array setting:
#   - With synthetic_array=True (default):
#     a shape: [num_rx, num_rx_ant, num_tx, num_tx_ant, num_paths, num_time_steps]
#     tau shape: [num_rx, num_tx, num_paths]  ← Note: NO antenna dimensions
#   - With synthetic_array=False:
#     Both have antenna dimensions

a_np, tau_np = paths.cir(out_type='numpy')

# Handle real/imag components if returned separately
if isinstance(a_np, tuple) and len(a_np) == 2:
    a_np = a_np[0] + 1j * a_np[1]  # Combine to complex array
elif not np.iscomplexobj(a_np):
    a_np = a_np.astype(np.complex128)

# Debug: Print shapes to understand tensor dimensions
print(f"Debug - a shape: {a_np.shape}")
print(f"Debug - tau shape: {tau_np.shape}")
print()

# Get number of paths from shape
num_paths = a_np.shape[4]  # Index 4 is num_paths dimension

print(f"Number of paths: {num_paths}")
print("\nPath Analysis:")
print("=" * 60)
print(f"{'Path':<8} {'Power (dB)':<15} {'Delay (ns)':<15}")
print("-" * 60)

for i in range(num_paths):
    # Extract complex amplitude for this path
    # Indexing: [rx=0, rx_ant=0, tx=0, tx_ant=0, path=i, time=0]
    a_complex = a_np[0, 0, 0, 0, i, 0]
    amplitude = np.abs(a_complex)
    
    # Convert to dB (20*log10 for field quantities)
    power_db = 20 * np.log10(amplitude) if amplitude > 0 else -np.inf
    
    # Extract delay in nanoseconds
    # CRITICAL: tau shape is [num_rx, num_tx, num_paths] with synthetic arrays
    # NOT [num_rx, num_rx_ant, num_tx, num_tx_ant, num_paths]
    delay_ns = float(tau_np[0, 0, i]) * 1e9
    
    print(f"{i:<8} {power_db:<15.1f} {delay_ns:<15.2f}")

print("-" * 60)
print("\nInterpretation:")
print("- Strongest path (lowest loss): Likely LOS through doorway or strong reflection")
print("- Weaker paths (<-80 dB): Likely through-wall transmission (highly attenuated)")
print("- Intermediate paths: Reflections, diffractions, or mixed interactions")

## Compare: With vs Without Refraction

See the difference when refraction (transmission through walls) is disabled:

In [ ]:
# Compute paths WITHOUT refraction (no transmission through walls)
solver_no_refraction = PathSolver(
    refraction=False,          # Disable transmission through materials
    specular_reflection=True,  # Keep reflections
    diffraction=True           # Keep diffraction
)
paths_no_refraction = solver_no_refraction(scene)

# Preview - should only show paths through doorway
scene.preview(paths=paths_no_refraction, clip_at=2.0)

# Get path count from CIR (using NumPy output)
a_no_ref_np, _ = paths_no_refraction.cir(out_type='numpy')

# Handle real/imag components if returned separately
if isinstance(a_no_ref_np, tuple) and len(a_no_ref_np) == 2:
    a_no_ref_np = a_no_ref_np[0] + 1j * a_no_ref_np[1]

num_paths_no_ref = a_no_ref_np.shape[4]

print(f"Paths WITH refraction (default): {num_paths}")
print(f"Paths WITHOUT refraction: {num_paths_no_ref}")
print("\nNote: Fewer paths without refraction = only through-doorway paths")

## Scene Object Details

List all objects with their materials and geometry:

In [ ]:
# Detailed object information with bounding box corners
print("Detailed Object Geometry:")
print("=" * 90)

for name, obj in scene.objects.items():
    bbox = obj.mi_mesh.bbox()
    material = obj.radio_material.name if obj.radio_material else "None"
    # Extract scalar values from DrJit types
    pos = obj.position
    px, py, pz = pos[0][0], pos[1][0], pos[2][0]
    min_x, min_y, min_z = bbox.min[0], bbox.min[1], bbox.min[2]
    max_x, max_y, max_z = bbox.max[0], bbox.max[1], bbox.max[2]
    
    print(f"\n{name}:")
    print(f"  Material: {material}")
    print(f"  Center:   ({px:.2f}, {py:.2f}, {pz:.2f})")
    print(f"  Min:      ({min_x:.2f}, {min_y:.2f}, {min_z:.2f})")
    print(f"  Max:      ({max_x:.2f}, {max_y:.2f}, {max_z:.2f})")
    print(f"  Size:     ({max_x - min_x:.2f} x {max_y - min_y:.2f} x {max_z - min_z:.2f})")

## Material Properties

Check ITU concrete material properties at the configured frequency:

In [ ]:
# Get material properties
concrete = scene.get("itu_concrete")

print("ITU Concrete Material Properties:")
print("=" * 50)
print(f"Frequency: {scene.frequency / 1e9:.2f} GHz")
print(f"Relative permittivity (εᵣ): {concrete.relative_permittivity}")
print(f"Conductivity (σ): {concrete.conductivity} S/m")
print(f"Thickness: {concrete.thickness} m")
print("\nITU-R P.2040-3 Model:")
print("  εᵣ = 5.24 × f^0 = 5.24 (frequency-independent)")
print("  σ = 0.0462 × f^0.7822 ≈ 0.15 S/m at 5.18 GHz")